In [6]:
import spacy
import os

from preprocessing.cleaning_utils import *

In [12]:
from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict
from datasets import load_metric


In [2]:
assessment_model_f = "/home/vs428/Documents/prodigy/assessment_model/model-best"
plan_model_f = "/home/vs428/Documents/prodigy/plan_subsection_model/model-best"

In [4]:
nlp_plan = spacy.load(plan_model_f, exclude="parser")
nlp_assess = spacy.load(assessment_model_f, exclude="parser")
# nlp_orig = spacy.load("en_core_web_trf")
# nlp.add_pipe("parser", source=nlp_orig, after="transformer")
# nlp.add_pipe("tagger", source=nlp_orig, after="parser")



In [193]:
tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
model = AutoModelForSequenceClassification.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext", num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cambridgeltl/SapBERT-from-PubMedBERT-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:

# Read MIMIC notes
# notes = pd.read_csv(cfg.data.mimic_data_dir + "NOTEEVENTS.csv")

# create hf Dataset
classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']

# instead we will use the raw text for now
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'Plan Subsection':Value("string"),
    "Relation":Value("string")
}) 

dataset = load_dataset("csv", data_files={
                            "train":"/home/vs428/project/n2c2/2022/Data/train.csv",
                            "valid":"/home/vs428/project/n2c2/2022/Data/dev.csv",
                        },
                       features=features)


Using custom data configuration default-b1948d86214b7517
Reusing dataset csv (/home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
test = ['51 yr old F with a history of 3V CAD, confirmed on C. cath during this\n   admission, EF of 40%, who is transferred to CCU for monitoring of\n   recurrent chest pain. Plan for CABG today to revascularize due to\n   3-vessel disease.',
       'Patient is a 58 year old male with history of atrial fibrillation on\n   coumadin, hyperlipidemia, hypertension, peripheral vascular disease,\n   s/p CVA who presented to [**Hospital3 847**] on [**2185-3-2**] with heart\n   failure exacerbation and atrial fibrillation with RVR, transferred due\n   to difficulty with diuresis [**2-7**] hypotension.']

In [ ]:
N = 1

In [48]:
print(test[N])

Patient is a 58 year old male with history of atrial fibrillation on
   coumadin, hyperlipidemia, hypertension, peripheral vascular disease,
   s/p CVA who presented to [**Hospital3 847**] on [**2185-3-2**] with heart
   failure exacerbation and atrial fibrillation with RVR, transferred due
   to difficulty with diuresis [**2-7**] hypotension.


In [49]:
doc = nlp_assess(test[N])

/gpfs/ysm/project/rtaylor/vs428/conda_envs/n2c2_env/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [50]:
doc.ents

In [85]:
dataset['train'] = dataset['train'].shard(num_shards=1000, index=0)
dataset['valid'] = dataset['train'].shard(num_shards=100, index=0)


In [64]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label, ent.label_)

atrial fibrillation 46 65 7482963781161052880 secondary_problem
hyperlipidemia 82 96 7482963781161052880 secondary_problem
hypertension 98 110 7482963781161052880 secondary_problem
peripheral vascular disease 112 139 7482963781161052880 secondary_problem
CVA 148 151 7482963781161052880 secondary_problem
heart
   failure exacerbation 212 241 8591029796264612076 primary_problem
atrial fibrillation with RVR 246 274 8591029796264612076 primary_problem
diuresis 314 322 3545868656138680645 primary_sign
hypotension 333 344 3545868656138680645 primary_sign


In [135]:
from functools import partial

In [166]:
test = dataset['valid'].map(partial(add_ner_plan, nlp=nlp_plan))

  0%|          | 0/1 [00:00<?, ?ex/s]

/gpfs/ysm/project/rtaylor/vs428/conda_envs/n2c2_env/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [170]:
test[0]['Assessment'], test[0]['Plan Subsection']

('51 yr old F with a history of 3V CAD, confirmed on C. cath during this\n   admission, EF of 40%, who is transferred to CCU for monitoring of\n   recurrent chest pain. Plan for CABG today to revascularize due to\n   3-vessel disease.',
 '# CORONARIES: Patient with 3 vessel disease on cath. Previously found\n   to be the case in [**2174**], but had been treated medically. On the floor,\n   patient with <problem>recurrent chest pain</problem> and <event_related_to_problem>EKG changes</event_related_to_problem>, <complication_related_to_problem>concerning for\n   further ischemia</complication_related_to_problem>. Chest pain has been controlled on nitro gtt. Patient\n   in CCU for monitoring; <event_related_to_problem>plan for CABG today</event_related_to_problem>.\n   - continue <event_related_to_problem>nitro gtt</event_related_to_problem>\n   - if has further CP, would uptitrate nitro, and contact [**Name2 (NI) 9339**] and\n   attending for potential balloon pump vs. urgent CABG\n   -

# Test Tokenizer after adding entity tags

In [148]:
tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
model = AutoModelForSequenceClassification.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext", num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cambridgeltl/SapBERT-from-PubMedBERT-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [183]:
tokenizer

PreTrainedTokenizerFast(name_or_path='cambridgeltl/SapBERT-from-PubMedBERT-fulltext', vocab_size=30522, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [195]:
spans = [x for x in nlp_plan.get_pipe("ner").labels if x.islower()] + [x for x in nlp_assess.get_pipe("ner").labels if x.islower()]

In [196]:
tokens = []
for span in spans:
    tokens.append("<" + span + ">")
    tokens.append("</" + span + ">")    

In [197]:
from tokenizers import AddedToken
tokens = [AddedToken(token) for token in tokens]

In [198]:
x = tokenizer.add_tokens(tokens)

In [199]:
x

16

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30538, 768)

In [205]:
tokenizer.decode(tokenizer(test[0]['Plan Subsection'])['input_ids']), tokenizer(test[0]['Plan Subsection'])

('[CLS] # coronaries : patient with 3 vessel disease on cath. previously found to be the case in [ * * 2174 * * ], but had been treated medically. on the floor, patient with <problem> recurrent chest pain </problem> and <event_related_to_problem> ekg changes </event_related_to_problem>, <complication_related_to_problem> concerning for further ischemia </complication_related_to_problem>. chest pain has been controlled on nitro gtt. patient in ccu for monitoring ; <event_related_to_problem> plan for cabg today </event_related_to_problem>. - continue <event_related_to_problem> nitro gtt </event_related_to_problem> - if has further cp, would uptitrate nitro, and contact [ * * name2 ( ni ) 9339 * * ] and attending for potential balloon pump vs. urgent cabg - appreciate ct [ * * doctor first name 91 * * ] recs - continue [ * * last name ( lf ) * * ], [ * * first name3 ( lf ) 119 * * ], ace - i, simvastatin [SEP]',
 {'input_ids': [2, 7, 5548, 5816, 30, 2774, 1956, 23, 8072, 2573, 1990, 6258, 

In [207]:
tokenizer.convert_tokens_to_ids("pain")

3594

In [22]:
import re 

def remove_mimic_deid(example):
    
    example['Assessment'] = re.sub(r"\[\*\*.*?\*\*\]", "", example['Assessment'])
    example['Plan Subsection'] = re.sub(r"\[\*\*.*?\*\*\]", "", example['Plan Subsection'])    
    
    return example

In [23]:
out = dataset.map(remove_mimic_deid)

  0%|          | 0/4633 [00:00<?, ?ex/s]

  0%|          | 0/597 [00:00<?, ?ex/s]

In [24]:
out['train'][123]['Assessment'], dataset['train'][123]['Assessment']

('Patient is a 58 year old male with history of atrial fibrillation on\n   coumadin, hyperlipidemia, hypertension, peripheral vascular disease,\n   s/p CVA who presented to  on  with heart\n   failure exacerbation and atrial fibrillation with RVR, transferred due\n   to difficulty with diuresis  hypotension.',
 'Patient is a 58 year old male with history of atrial fibrillation on\n   coumadin, hyperlipidemia, hypertension, peripheral vascular disease,\n   s/p CVA who presented to [**Hospital3 847**] on [**2185-3-2**] with heart\n   failure exacerbation and atrial fibrillation with RVR, transferred due\n   to difficulty with diuresis [**2-7**] hypotension.')